In [1]:
# ensure cwd is project root
import os
root = r"C:\Users\Dell\Documents\Customer-Churn-Prediction"
if os.getcwd() != root:
    os.chdir(root)
print("cwd:", os.getcwd())


cwd: C:\Users\Dell\Documents\Customer-Churn-Prediction


In [2]:
# dataset check
from pathlib import Path
p = Path("data")/"raw"
print("expected dataset path:", p.resolve())
print("exists?", p.exists())
if p.exists():
    files = list(p.iterdir())
    print("files in data/raw:")
    for f in files:
        print("-", repr(f.name), f.stat().st_size, "bytes")
else:
    print("data/raw not found. Create data/raw and place churn_modelling.csv inside.")

expected dataset path: C:\Users\Dell\Documents\Customer-Churn-Prediction\Data\raw
exists? True
files in data/raw:
- 'Churn_Modelling.csv' 684858 bytes


In [3]:
# quick load first rows & shape
import pandas as pd
from pathlib import Path

data_path = Path("data")/"raw"/"churn_modelling.csv"
if not data_path.exists():
    raise FileNotFoundError(f"{data_path} not found. Put dataset at this path.")
df = pd.read_csv(data_path)
print("rows, cols:", df.shape)
display(df.head(5))

rows, cols: (10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
